## Add needed libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## We are loading two datasets: 'listings.csv' and 'calendar.csv'. 

The 'listings.csv' file contains detailed information about various listings, while the 'calendar.csv' file contains pricing and availability information for each listing. 
Unfortunate price for all the different date here is the same so it will not allow us to provide analysis from this perspectives

We then clean the 'price' columns in both datasets by removing dollar signs and commas, and converting the values to float type for easier numerical operations. 
Next, we calculate the average price for available listings and all listings, grouping by 'listing_id'. This helps us understand the pricing trends for available and all listings separately. 

Finally, we delete the 'price_df' to free up memory.

In [24]:
listing_df = pd.read_csv('data/listings.csv')
price_df = pd.read_csv('data/calendar.csv')

#removing dollar signs and commas, and converting the values to float for both dataframes
listing_df['price'] = listing_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
price_df['price'] = price_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# this step is not necessary for our data, but it is a good practice to get mean  for availble prices
# and mean for all prices 
grouped_avail_price_df = price_df[price_df['available'] == 't'].groupby('listing_id')['price'].mean().reset_index()
grouped_avail_price_df.rename(columns={'price': 'average_price','listing_id': 'id'}, inplace=True)

grouped_all_price_df = price_df.groupby('listing_id')['price'].mean().reset_index()
grouped_all_price_df.rename(columns={'price': 'average_price','listing_id': 'id'}, inplace=True)

del price_df





<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
C:\Users\A93074843\AppData\Local\Temp\ipykernel_13152\1063296803.py:5: SyntaxWarning: invalid escape sequence '\$'
  listing_df['price'] = listing_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
C:\Users\A93074843\AppData\Local\Temp\ipykernel_13152\1063296803.py:6: SyntaxWarning: invalid escape sequence '\$'
  price_df['price'] = price_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)


## Merging DataFrames and Handling Missing Values

In this step, we merge the `listing_df` DataFrame with the `grouped_avail_price_df` and `grouped_all_price_df` DataFrames to incorporate the average prices for available and all listings, respectively. 

We then handle missing values in the `price` column by filling them with the corresponding average prices. This ensures that all listings have a price value, which is crucial for further analysis. 

Finally, we drop the `average_price` column from the merged DataFrames as it is no longer needed.


In [25]:
# try to merge prices to the listing_df where it's missing
# results how much prices we have before and after merge are going to be printed
print (f' Number of rows before merge {listing_df.shape[0]}')
print (f'number of missed price: {listing_df['price'].isnull().sum()}')
avail_merged_df = pd.merge(listing_df, grouped_avail_price_df, on='id', how='left')
avail_merged_df['price'] = avail_merged_df['price'].fillna(avail_merged_df['average_price'])
avail_merged_df.drop(columns=['average_price'], inplace=True)
print (f' Number of rows after merge {avail_merged_df.shape[0]}')
print (f'number of missed price: {avail_merged_df['price'].isnull().sum()}')


all_merged_df = pd.merge(listing_df, grouped_all_price_df, on='id', how='left')
all_merged_df['price'] = all_merged_df['price'].fillna(all_merged_df['average_price'])
all_merged_df.drop(columns=['average_price'], inplace=True)
print (f' Number of rows after merge {all_merged_df.shape[0]}')
print (f'number of missed price: {all_merged_df['price'].isnull().sum()}')


 Number of rows before merge 7281
number of missed price: 2554
 Number of rows after merge 7281
number of missed price: 2147
 Number of rows after merge 7281
number of missed price: 0


## Filtering and Grouping Data

In this step, we create a new column `accommodates_grouped` in both `avail_merged_df` and `all_merged_df` DataFrames to group listings that accommodate 6 or more people into a single category labeled '6+'. 

We also remove outliers by filtering out listings with a price greater than 1000 for one night. This helps in focusing the analysis on more typical listings and avoiding skewed results due to extreme values.

In [26]:
# Create a new column that groups accommodates values of 5 and more into a single category
avail_merged_df['accommodates_grouped'] = avail_merged_df['accommodates'].apply(lambda x: '6+' if x >= 6 else str(x))
all_merged_df['accommodates_grouped'] = all_merged_df['accommodates'].apply(lambda x: '6+' if x >= 6 else str(x))

#Remove outliers we don't want to consider (price > 1000 for one night)
avail_merged_df = avail_merged_df[all_merged_df['price'] <= 1000]
all_merged_df = all_merged_df[all_merged_df['price'] <= 1000]



## Haversine Function

The Haversine function is used to calculate the great-circle distance between two points on the Earth's surface, given their latitude and longitude. This formula accounts for the spherical shape of the Earth and provides an accurate distance measurement.

The function takes four parameters:
- `lat1`: Latitude of the first point.
- `lon1`: Longitude of the first point.
- `lat2`: Latitude of the second point.
- `lon2`: Longitude of the second point.

The function returns the distance between the two points in kilometers.

In [27]:
# Coordinates of the center of Munich
munich_center = (48.137154, 11.576124)

# Function to calculate the Haversine distance
'''
This function calculates the Haversine distance between two points on the Earth given their latitude and longitude in decimal degrees.
Inputs:
- lat1, lon1: latitude and longitude of point 1 in decimal degrees
- lat2, lon2: latitude and longitude of point 2 in decimal degrees
Output:
- distance between the two points in kilometers
Haversine formula:
a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
c = 2 ⋅ atan2( √a, √(1−a) )

'''
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


## Calculating Distance from City Center

In this step, we calculate the distance of each listing from the center of Munich using the Haversine formula. The Haversine formula is used to determine the distance between two points on the Earth's surface given their latitude and longitude. 

We then categorize these distances into different ranges: '0-3 km', '3-5 km', '5-10 km', and '10+ km'. This categorization helps in analyzing the distribution of listings based on their proximity to the city center. 

The calculated distances and their respective categories are added as new columns `distance_from_center` and `distance_category` in both `avail_merged_df` and `all_merged_df` DataFrames.

In [28]:
# Categorize the listings into different distance ranges
bins = [0, 3, 5, 10, np.inf]
labels = ['0-3 km', '3-5 km', '5-10 km', '10+ km']

# Calculate the distance of each listing from the center of Munich and categorize it
avail_merged_df['distance_from_center'] = avail_merged_df.apply(lambda row: haversine(row['latitude'], row['longitude'], munich_center[0], munich_center[1]), axis=1)
avail_merged_df['distance_category'] = pd.cut(avail_merged_df['distance_from_center'], bins=bins, labels=labels)

all_merged_df['distance_from_center'] = all_merged_df.apply(lambda row: haversine(row['latitude'], row['longitude'], munich_center[0], munich_center[1]), axis=1)
all_merged_df['distance_category'] = pd.cut(all_merged_df['distance_from_center'], bins=bins, labels=labels)


## Saving Processed DataFrames

In the final step, we save the processed DataFrames `avail_merged_df` and `all_merged_df` to CSV files. These files can be used for further analysis or shared with others for their own analysis. The files are saved as `listings_avail_processed.csv` and `listings_all_processed.csv` respectively.

In [29]:
avail_merged_df.to_csv('data/listings_avail_processed.csv', index=False)
all_merged_df.to_csv('data/listings_all_processed.csv', index=False)